# MDA HW3 Question 3

#### Importing packages and install pre requirements

In [1]:
import os
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 8.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a5e03a4284d9e05bebe1700b4d057f10437664304798cd0be57cbf529cfc91ed
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The followin

## Initializing Spark and read data

In [2]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,TimestampType
from pyspark.sql.functions import col,current_timestamp,to_date,hour,dayofweek
spark = SparkSession \
    .builder \
    .appName("Spark_Processor") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

schema = StructType([ \
        StructField("DEVICE_CODE", IntegerType(), True), 
        StructField("SYSTEM_ID",IntegerType(),True), \
        StructField("ORIGINE_CAR_KEY",StringType(),True), \
        StructField("FINAL_CAR_KEY",StringType(),True), \
        StructField("CHECK_STATUS_KEY", IntegerType(), True), \
        StructField("COMPANY_ID", StringType(), True), \
        StructField("PASS_DAY_TIME", TimestampType(), True)
    ])

23/01/18 01:22:10 WARN Utils: Your hostname, soltani-PE62-8RC resolves to a loopback address: 127.0.1.1; using 172.27.172.133 instead (on interface wlo1)
23/01/18 01:22:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/18 01:22:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Part a)
#### At first we do the process for SampleData.csv and after that we do all the process for the original dataset.

In [2]:
df=spark.read.csv('SampleData.csv',header=True,schema=schema)
df.show(10)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|  100700853|      283|        9016704|      9016704|               5|       264|2022-01-08 00:00:39|
|     230204|       81|        9017538|      9017538|               5|       161|2022-01-08 00:38:48|
|     631357|       81|        9017538|      9017538|               5|       161|2022-01-08 00:13:50|
|     900164|      283|        9035020|      9035020|               5|       264|2022-01-08 00:27:38|
|     900149|      283|        9020900|      9020900|               5|       264|2022-01-08 00:40:08|
|     900246|      283|        9021792|      9021792|               5|       264|2022-01-08 00:55:51|
|  100700871|      283|        9021792|      9021792|               5|       264|2

#### By using map, we make each row of the df to car license plate and time of their passing. We put the key of the rdd the code of the device which shows the pathway that is passed. The result rdd has a row for each of pathway in each day. 

In [3]:
rdd = df.rdd.map(lambda x: ((x['FINAL_CAR_KEY'], x['PASS_DAY_TIME'].date()),  x['DEVICE_CODE']))
rdd.take(10)

[(('9016704', datetime.date(2022, 1, 8)), 100700853),
 (('9017538', datetime.date(2022, 1, 8)), 230204),
 (('9017538', datetime.date(2022, 1, 8)), 631357),
 (('9035020', datetime.date(2022, 1, 8)), 900164),
 (('9020900', datetime.date(2022, 1, 8)), 900149),
 (('9021792', datetime.date(2022, 1, 8)), 900246),
 (('9021792', datetime.date(2022, 1, 8)), 100700871),
 (('9021792', datetime.date(2022, 1, 8)), 900235),
 (('9038947', datetime.date(2022, 1, 8)), 100701059),
 (('9024221', datetime.date(2022, 1, 8)), 631765)]

#### If we do group_by for the above rdd, the rdd will have all the pathways for each car in each specific day in a list. In this part we use set to not have repeated itemset. This approach will not have huge effect on the problem and will not change the main results.

In [4]:
path_rdd = rdd.groupByKey().mapValues(set)
path_rdd.take(10)

[(('9020900', datetime.date(2022, 1, 8)), {900149}),
 (('9024374', datetime.date(2022, 1, 8)),
  {103002, 631366, 631829, 900215, 900234, 900251, 22009972}),
 (('9153235', datetime.date(2022, 1, 8)), {900142, 900156, 900199}),
 (('9160867', datetime.date(2022, 1, 8)),
  {900142, 900199, 22010039, 100700853}),
 (('9234928', datetime.date(2022, 1, 8)), {137, 212802}),
 (('9332040', datetime.date(2022, 1, 8)),
  {200301,
   205802,
   631354,
   900212,
   900215,
   900221,
   900223,
   900225,
   900256,
   900259,
   900273,
   900277,
   100700826}),
 (('9385850', datetime.date(2022, 1, 8)), {900212, 900222, 900244, 100700839}),
 (('9433390', datetime.date(2022, 1, 8)), {145}),
 (('9423518', datetime.date(2022, 1, 8)), {900101}),
 (('9435137', datetime.date(2022, 1, 8)), {900101, 900156, 900211, 100701090})]

#### Now we obtain the size of the rdd.

In [5]:
path_rdd.count()

1479063

# Part b)


#### For some applications and process we don't need to have date and license plate of cars. So we make a new rdd without them and work with it whenever we need.

In [6]:
import numpy as np
import math

path_rdd_without_key = path_rdd.map(lambda x: x[1])
path_rdd_without_key.take(10)

[{900149},
 {103002, 631366, 631829, 900215, 900234, 900251, 22009972},
 {900142, 900156, 900199},
 {900142, 900199, 22010039, 100700853},
 {137, 212802},
 {200301,
  205802,
  631354,
  900212,
  900215,
  900221,
  900223,
  900225,
  900256,
  900259,
  900273,
  900277,
  100700826},
 {900212, 900222, 900244, 100700839},
 {145},
 {900101},
 {900101, 900156, 900211, 100701090}]

#### In the below cell, a sample path is selected for this part to get compared to pathways of the main rdd to find the most similar pathways to that.

In [7]:
sample_pathway = path_rdd_without_key.take(20)[2]
sample_pathway

{900142, 900156, 900199}

#### In the next cell the "similarity" function is defined which it gets a pathway as input and return similarity between that pathway to sample pathway. It subscripts the sample pathway with the input pathway and it calculates the cosine distance between the sample pathway and the input pathway by dividing the length of intersection of them to sqrt of the multiple of that input and sample pathway. The function is defined in below cell.

In [9]:
def similarity(input_pathway):
    pathway = input_pathway[1]
    distance = len(sample_pathway.intersection(pathway)) / math.sqrt((len(pathway)*len(sample_pathway)))

    return (input_pathway, distance)


#### In the below cell, the most 5 similar pathways to the sample pathway is collected. In each tuple, the first parameter is license plate and then is datetime and after that is pathway and the last one is the amount of similarity. The result is shown in below cell.

In [9]:
path_rdd.map(similarity).sortBy(lambda x: x[1], ascending=False).take(5)

[((('9153235', datetime.date(2022, 1, 8)), {900142, 900156, 900199}), 1.0),
 ((('103814654', datetime.date(2022, 1, 8)), {900142, 900156, 900199}), 1.0),
 ((('92099066', datetime.date(2022, 1, 8)), {900142, 900156, 900199}), 1.0),
 ((('100913501', datetime.date(2022, 1, 8)), {900142, 900156, 900199}), 1.0),
 ((('11251721', datetime.date(2022, 1, 8)), {900142, 900156, 900199}), 1.0)]

# Part c)

#### In the next cell "generate_vectors" function is define which it gets an input as number of hash functions and then it generate n vectors of 1 and -1. These vectors are made as dictionary because in the next parts, intersections will be calculated much easier and faster. The function is shown in below cell.

In [10]:
import random

def generate_vectors(n):
    all_path = dict.fromkeys(path_rdd_without_key.flatMap(lambda x: x).distinct().collect(), 1)

    vectors = [None]*n
    for i in range(n):
        v = all_path.copy()
        for key in all_path:
            v[key] = [1,-1][random.randint(0, 1)]

        vectors[i] = v

    return vectors

#### In the next cell we set the n, AND and OR parameters. "AND" and "OR" shows the width and number of hyperplanes respectively. It is also abvious that multiple of "AND" and "OR" is equal to n. Also we call generate_vectors function to generate the random vectors.

In [11]:
n = 500
AND = 25
OR = 20
vectors = generate_vectors(n)
vectors[1]

{900156: -1,
 900264: -1,
 22010040: -1,
 22010112: 1,
 100700820: -1,
 631368: -1,
 100700880: -1,
 900228: 1,
 168: 1,
 631776: 1,
 144: -1,
 100701252: -1,
 22010088: -1,
 100701096: 1,
 100701156: 1,
 100701132: -1,
 631608: 1,
 631356: 1,
 900216: 1,
 100700964: -1,
 631488: -1,
 22010052: 1,
 900180: 1,
 900132: -1,
 108: 1,
 156: 1,
 100701144: 1,
 635556: -1,
 100701120: -1,
 631644: 1,
 22009920: -1,
 22010100: -1,
 900144: 1,
 900120: -1,
 631800: -1,
 631860: -1,
 120: -1,
 711012: -1,
 631716: 1,
 900168: -1,
 635712: 1,
 631932: -1,
 631560: -1,
 100700928: 1,
 1001016: 1,
 635616: 1,
 1001004: -1,
 605508: -1,
 631884: 1,
 22009848: 1,
 631116: 1,
 1001040: 1,
 631848: -1,
 631692: 1,
 22009812: -1,
 710604: 1,
 22009836: -1,
 1001028: -1,
 631980: -1,
 631140: -1,
 110220: 1,
 22010136: 1,
 631896: 1,
 900277: 1,
 145: 1,
 631357: -1,
 100700881: -1,
 631765: 1,
 22010053: 1,
 631633: 1,
 100700965: -1,
 205201: 1,
 900241: 1,
 900217: -1,
 100700857: -1,
 100700929: -1,

#### In the next cell the "min_hash" is defined which gets a list of pathways as input and return signature of that. it intersect the vector of this list to all of the random vectors which are generated in the last part, for each vector. As the lists are small rather than the pathways, in this approach we just consider random vectors which exist on the main list and we calculate the intersection easily. If the result of intersection is positive, the value of the hash will be equal to 1 and if the result is negative the value of the hash will be equal to 0.

In [11]:
def min_hash(path):
    column = path[1]
    n = len(vectors)
    signature = [1]*n
    for i, v in enumerate(vectors):
        sum = 0

        for key in column:
            sum += v[key]

        if sum < 0:
            signature[i] = 0

    return (path[0], path[1], signature)


#### Now with using the above function and a map, we calculate the signature of each pathway.

In [13]:
signatures = path_rdd.map(min_hash)
signatures.take(1)

[(('9020900', datetime.date(2022, 1, 8)),
  {900149},
  [1,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   0,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   0,
   1

#### The below function gets a signature as input and generate a tuple for each element of the signature which keep all information about hash number, the value of the hash and the primary information. The reason of keeping this information is that we can show the results better. The combination of the hash number and the hash value gives us a specific basket of a hash. We use this in few next parts.

In [14]:
def hash_band(sign):
    signature = sign[2]
    output = []
    for i in range(OR):
        band = signature[i*AND:(i+1)*AND]
        band_binary = ''.join(str(digit) for digit in band)
        band_hash = int(band_binary, 2)
        output.append(
            (sign[0], i, band_hash, sign[1], sign[2])
        )

    return output

#### Now we get tuples by using the above function in the below cell.

In [15]:
hash_baskets = signatures.flatMap(hash_band)
hash_baskets.take(1)

[(('9020900', datetime.date(2022, 1, 8)),
  0,
  20051957,
  {900149},
  [1,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   0,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,

#### In the next cell we do all of the process which are explained above for our sample pathway.

In [16]:
sample_signature = min_hash(('sample car', sample_pathway))
print(sample_signature)

sample_baskets = hash_band(sample_signature)
print(sample_baskets[0])


('sample car', {900156, 900142, 900199}, [0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0

#### In this part which is actually the most important part of the code, we collect the final candidates. For this, as we understood in above parts, The combination of the hash number and the hash value gives us a specific basket of a hash. Now for each of these hashes, we only keep baskests which the hash of the sample pathway exists on that basket. It means that, the remaining pathways has similar hash with the pathway in at least one of the hyperplanes. The result is shown in below cells.

In [17]:
candidates = hash_baskets.filter(lambda x: any([ (x[1]==i) and (x[2] == sample_baskets[i][2]) for i in range(OR)]))
candidates.take(1)

[(('9153235', datetime.date(2022, 1, 8)),
  0,
  2762006,
  {900142, 900156, 900199},
  [0,
   0,
   0,
   1,
   0,
   1,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   0,
   0,
   1,
   0,
   1,
   0,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   0,
   0,
   1,
   0,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
  

#### In this part we use groupBy and map to remove repeated pathways. The remaining pathways which are the final candidates are specific and unique. Then we count the number of unique candidates.

In [18]:
candidates_unique = candidates.map(lambda x: (x[0], (x[1],x[2],x[3],x[4]) )).groupByKey().mapValues(lambda x: list(x)[0])
candidates_unique.take(1)

[(('9153235', datetime.date(2022, 1, 8)),
  (0,
   2762006,
   {900142, 900156, 900199},
   [0,
    0,
    0,
    1,
    0,
    1,
    0,
    1,
    0,
    0,
    0,
    1,
    0,
    0,
    1,
    0,
    1,
    0,
    0,
    0,
    1,
    0,
    1,
    1,
    0,
    0,
    0,
    0,
    0,
    1,
    1,
    1,
    0,
    1,
    0,
    1,
    1,
    1,
    1,
    0,
    0,
    1,
    0,
    0,
    1,
    0,
    0,
    0,
    0,
    1,
    1,
    0,
    1,
    0,
    0,
    0,
    1,
    0,
    1,
    0,
    0,
    1,
    0,
    1,
    0,
    1,
    0,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    0,
    0,
    1,
    0,
    0,
    0,
    0,
    1,
    1,
    1,
    1,
    1,
    1,
    0,
    0,
    1,
    1,
    1,
    1,
    1,
    1,
    0,
    1,
    0,
    1,
    1,
    0,
    1,
    1,
    1,
    0,
    1,
    1,
    0,
    1,
    1,
    0,
    1,
    1,
    0,
    1,
    1,
    1,
    0,
    1,
    0,
    0,
    1,
    0,
    0,
    1,
    0,
    0,
 

In [19]:
candidates_unique_count = candidates_unique.count()
candidates_unique_count

15

In [20]:
final_candidates_pathways = candidates.map(lambda x: (x[0], x[3])).groupByKey().mapValues(lambda x: list(x)[0])
final_candidates_count = final_candidates_pathways.count()
final_candidates_count

15

In [21]:
final_candidates_pathways.take(final_candidates_count)

[(('9153235', datetime.date(2022, 1, 8)), {900142, 900156, 900199}),
 (('84810729', datetime.date(2022, 1, 8)),
  {900142, 900156, 900199, 900212, 100700841}),
 (('103814654', datetime.date(2022, 1, 8)), {900142, 900156, 900199}),
 (('8821973', datetime.date(2022, 1, 8)),
  {900142, 900156, 900199, 900212, 900244}),
 (('92099066', datetime.date(2022, 1, 8)), {900142, 900156, 900199}),
 (('65668312', datetime.date(2022, 1, 8)),
  {900132, 900142, 900156, 900199, 100700841}),
 (('100913501', datetime.date(2022, 1, 8)), {900142, 900156, 900199}),
 (('93846333', datetime.date(2022, 1, 8)),
  {900142, 900156, 900199, 900212, 900244}),
 (('86947187', datetime.date(2022, 1, 8)),
  {900142, 900156, 900199, 22010095, 100701090}),
 (('11251721', datetime.date(2022, 1, 8)), {900142, 900156, 900199}),
 (('7660802', datetime.date(2022, 1, 8)), {900142, 900156, 900199}),
 (('8086190', datetime.date(2022, 1, 8)), {900142, 900156, 900199}),
 (('90846416', datetime.date(2022, 1, 8)), {900142, 900156, 9

# Final Results and Parameters for SampleData
#### Actually as the number of hyperplane increases, we can decrease false negative and false positive. With testing different parameters, I figured out that the 25 number of "AND" and 20 number of "OR" would have the best result.

# Now we do all of the above process for the original data, TrafficData.csv
#### As the above cells are defined completely, We just run the cells for TrafficData and observe the results. The reuslts are shown in below cell.

In [3]:
df_traffic=spark.read.csv('TrafficData.csv',header=True,schema=schema)
df_traffic.show(10)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|   22010122|      284|       97955760|     64111706|               7|       161|2022-01-10 08:58:02|
|   22010047|      284|       34111333|     34111333|               6|       161|2022-01-10 08:09:03|
|     206701|       81|       33797174|     64111706|               7|       161|2022-01-10 07:50:48|
|   22010053|      284|       33845128|     33845128|               6|       161|2022-01-10 10:53:24|
|     900214|      283|       17144642|     64111706|               7|       264|2022-01-10 08:12:33|
|     631357|       81|       34136566|     34136566|               6|       161|2022-01-10 09:50:00|
|     900212|      283|       34109889|     34109889|               6|       264|2

In [4]:
rdd_traffic = df_traffic.rdd.map(lambda x: ((x['FINAL_CAR_KEY'], x['PASS_DAY_TIME'].date()),  x['DEVICE_CODE']))
rdd_traffic.take(10)

[(('64111706', datetime.date(2022, 1, 10)), 22010122),
 (('34111333', datetime.date(2022, 1, 10)), 22010047),
 (('64111706', datetime.date(2022, 1, 10)), 206701),
 (('33845128', datetime.date(2022, 1, 10)), 22010053),
 (('64111706', datetime.date(2022, 1, 10)), 900214),
 (('34136566', datetime.date(2022, 1, 10)), 631357),
 (('34109889', datetime.date(2022, 1, 10)), 900212),
 (('34109889', datetime.date(2022, 1, 10)), 900244),
 (('64111706', datetime.date(2022, 1, 10)), 900266),
 (('33797157', datetime.date(2022, 1, 10)), 900126)]

In [5]:
path_rdd_traffic = rdd_traffic.groupByKey().mapValues(set)
path_rdd_traffic.take(10)

[(('34136566', datetime.date(2022, 1, 10)), {631357, 631829, 900222}),
 (('33948272', datetime.date(2022, 1, 10)), {100701119}),
 (('33880146', datetime.date(2022, 1, 10)),
  {631633,
   631634,
   900101,
   900102,
   900199,
   900207,
   900208,
   900211,
   900212,
   900225,
   900244,
   900268,
   900269,
   900273,
   100700839,
   100701092}),
 (('33875661', datetime.date(2022, 1, 10)), {22010094}),
 (('33856705', datetime.date(2022, 1, 10)),
  {115, 631634, 631832, 900255, 900269, 100700853}),
 (('33880797', datetime.date(2022, 1, 10)), {145, 211301, 900101, 100700864}),
 (('33879718', datetime.date(2022, 1, 10)), {100700839}),
 (('33889314', datetime.date(2022, 1, 10)),
  {631368, 631765, 631829, 900164, 900214, 900222, 900228, 100701152}),
 (('7800904', datetime.date(2022, 1, 10)),
  {900134, 900223, 900225, 900268, 100700853, 100701100}),
 (('21792974', datetime.date(2022, 1, 12)),
  {209103, 900167, 900247, 100700804, 100700834})]

In [6]:
path_rdd_traffic.count()

10811026

In [7]:
path_rdd_traffic_without_key = path_rdd_traffic.map(lambda x: x[1])
path_rdd_traffic_without_key.take(10)

[{631357, 631829, 900222},
 {100701119},
 {631633,
  631634,
  900101,
  900102,
  900199,
  900207,
  900208,
  900211,
  900212,
  900225,
  900244,
  900268,
  900269,
  900273,
  100700839,
  100701092},
 {22010094},
 {115, 631634, 631832, 900255, 900269, 100700853},
 {145, 211301, 900101, 100700864},
 {100700839},
 {631368, 631765, 631829, 900164, 900214, 900222, 900228, 100701152},
 {900134, 900223, 900225, 900268, 100700853, 100701100},
 {209103, 900167, 900247, 100700804, 100700834}]

In [8]:
sample_pathway_traffic = path_rdd_traffic_without_key.take(10)[5]
sample_pathway_traffic

{145, 211301, 900101, 100700864}

In [9]:
import math
def similarity_traffic(input_pathway):
    pathway = input_pathway[1] 
    distance = len(sample_pathway_traffic.intersection(pathway)) / math.sqrt((len(pathway)*len(sample_pathway_traffic)))

    return (input_pathway, distance)

In [10]:
path_rdd_traffic.map(similarity_traffic).sortBy(lambda x: x[1], ascending=False).take(5)

[((('33880797', datetime.date(2022, 1, 10)), {145, 211301, 900101, 100700864}),
  1.0),
 ((('53554225', datetime.date(2022, 1, 13)), {145, 211301, 900101, 100700864}),
  1.0),
 ((('30176198', datetime.date(2022, 1, 14)), {145, 211301, 900101, 100700864}),
  1.0),
 ((('21275716', datetime.date(2022, 1, 14)), {145, 211301, 900101, 100700864}),
  1.0),
 ((('15801717', datetime.date(2022, 1, 13)), {145, 211301, 900101, 100700864}),
  1.0)]

#### In this part we just change number of n, "AND" and "OR" to get better result.

In [11]:
import random

def generate_vectors(n):
    all_path = dict.fromkeys(path_rdd_traffic_without_key.flatMap(lambda x: x).distinct().collect(), 1)

    vectors = [None]*n
    for i in range(n):
        v = all_path.copy()
        for key in all_path:
            v[key] = [1,-1][random.randint(0, 1)]

        vectors[i] = v
        
    return vectors

In [12]:
n = 2000
AND = 50
OR = 40
vectors = generate_vectors(n)
vectors[1]

{900102: 1,
 900228: 1,
 900214: 1,
 213402: -1,
 900256: -1,
 22010058: 1,
 900242: 1,
 100701090: -1,
 22010072: -1,
 900158: 1,
 22010044: -1,
 22010086: -1,
 100700894: 1,
 168: 1,
 22009834: 1,
 126: -1,
 900130: -1,
 22010100: -1,
 100701146: -1,
 100700978: 1,
 112: -1,
 22009918: 1,
 100701132: -1,
 635614: -1,
 100700964: -1,
 810110: -1,
 210602: -1,
 100701174: -1,
 100700880: -1,
 100700838: -1,
 900172: -1,
 154: 1,
 631610: -1,
 900144: 1,
 22010128: 1,
 140: -1,
 635572: 1,
 1001028: 1,
 22010030: 1,
 631358: 1,
 212002: 1,
 635712: -1,
 22009806: 1,
 801710: -1,
 631932: 1,
 631960: -1,
 22009960: -1,
 635516: -1,
 1001042: -1,
 230202: 1,
 631666: 1,
 100701160: -1,
 200802: -1,
 631120: 1,
 22009848: 1,
 631694: 1,
 100700950: 1,
 711018: -1,
 631890: 1,
 631848: 1,
 900270: -1,
 1001014: 1,
 635502: 1,
 710402: -1,
 711004: -1,
 22010114: -1,
 100701119: -1,
 100700839: -1,
 100700853: -1,
 631765: -1,
 900243: -1,
 155: -1,
 900215: 1,
 22010031: 1,
 100700881: 1,
 

In [13]:
def min_hash(path):
    column = path[1] 
    n = len(vectors)
    signature = [1]*n
    for i, v in enumerate(vectors):
        sum = 0
        
        for key in column:
            sum += v[key]

        if sum < 0:
            signature[i] = 0
            
    return (path[0], path[1], signature)


In [14]:
signatures_traffic = path_rdd_traffic.map(min_hash)
signatures_traffic.take(1)

[(('10293971', datetime.date(2022, 1, 8)),
  {100700847, 100701147},
  [0,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
 

In [15]:
def hash_band(sign):
    signature = sign[2]
    output = []
    for i in range(OR):
        band = signature[i*AND:(i+1)*AND]
        band_binary = ''.join(str(digit) for digit in band)
        band_hash = int(band_binary, 2)
        output.append(
            (sign[0], i, band_hash, sign[1], sign[2])
        )

    return output

In [16]:
hash_baskets_traffic = signatures_traffic.flatMap(hash_band)
hash_baskets_traffic.take(1)

[(('8992838', datetime.date(2022, 1, 13)),
  0,
  1086315200421879,
  {900155, 900183, 100700804, 100700834},
  [1,
   1,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
  

In [17]:
sample_signature_traffic = min_hash(('sample car for original data', sample_pathway_traffic))
print(sample_signature_traffic)

sample_baskets_traffic = hash_band(sample_signature_traffic)
print(sample_baskets_traffic[0])

('sample car for original data', {100700864, 145, 900101, 211301}, [1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,

In [18]:
candidates_traffic = hash_baskets_traffic.filter(lambda x: any([ (x[1]==i) and (x[2] == sample_baskets_traffic[i][2]) for i in range(OR)]))
candidates_traffic.take(1)

[(('30176198', datetime.date(2022, 1, 14)),
  0,
  979791556365311,
  {145, 211301, 900101, 100700864},
  [1,
   1,
   0,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   0,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   0,
   1,
   0,
   0,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
  

In [19]:
candidates_unique_traffic = candidates_traffic.map(lambda x: (x[0], (x[1],x[2],x[3],x[4]) )).groupByKey().mapValues(lambda x: list(x)[0])

In [20]:
final_candidates_pathways_traffic = candidates_traffic.map(lambda x: (x[0], x[3])).groupByKey().mapValues(lambda x: list(x)[0])

In [24]:
final_candidates_pathways_traffic_count = final_candidates_pathways_traffic.count()
final_candidates_pathways_traffic_count

28

In [25]:
final_candidates_pathways_traffic.take(final_candidates_pathways_traffic_count)

[(('33880797', datetime.date(2022, 1, 10)), {145, 211301, 900101, 100700864}),
 (('53554225', datetime.date(2022, 1, 13)), {145, 211301, 900101, 100700864}),
 (('30176198', datetime.date(2022, 1, 14)), {145, 211301, 900101, 100700864}),
 (('21275716', datetime.date(2022, 1, 14)), {145, 211301, 900101, 100700864}),
 (('15801717', datetime.date(2022, 1, 13)), {145, 211301, 900101, 100700864}),
 (('7972126', datetime.date(2022, 1, 14)), {145, 211301, 900101, 100700864}),
 (('14207373', datetime.date(2022, 1, 14)), {145, 211301, 900101, 100700864}),
 (('53679519', datetime.date(2022, 1, 8)), {145, 211301, 900101, 100700864}),
 (('9664169', datetime.date(2022, 1, 8)), {145, 211301, 900101, 100700864}),
 (('7752774', datetime.date(2022, 1, 14)), {145, 211301, 900101, 100700864}),
 (('7982725', datetime.date(2022, 1, 8)), {145, 211301, 900101, 100700864}),
 (('7965292', datetime.date(2022, 1, 8)), {145, 211301, 900101, 100700864}),
 (('13301346', datetime.date(2022, 1, 8)), {145, 211301, 9001